## Importing Libraries

In [1]:
# pytorch
import torch

# huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

## Hugging Face

In [2]:
model_name = "Waktaverse-Llama-2-KO-7B-Instruct"  # ADD YOUR MODEL NAME HERE
username = "PathFinderKR"  # ADD YOUR USERNAME HERE
repo_id = f"{username}/{model_name}"  # repository id

## Device

In [3]:
# Device setup
device = (
    "cuda:0" if torch.cuda.is_available() else # Nvidia GPU
    "mps" if torch.backends.mps.is_available() else # Apple Silicon GPU
    "cpu"
)
print(f"Device = {device}")

Device = cuda:0


## Merge

In [4]:
tokenizer_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [5]:
# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# Merge and unload
model = PeftModel.from_pretrained(base_model, model_name)
model = model.merge_and_unload()

## Upload

In [7]:
tokenizer.push_to_hub(
    repo_id=repo_id,
    use_temp_dir=False
)
model.push_to_hub(
    repo_id= repo_id,
    use_temp_dir=False
)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PathFinderKR/Waktaverse-Llama-2-KO-7B-Instruct/commit/31d1b5868b4a3ea2f5c4812491883a97f6b87219', commit_message='Upload LlamaForCausalLM', commit_description='', oid='31d1b5868b4a3ea2f5c4812491883a97f6b87219', pr_url=None, pr_revision=None, pr_num=None)